To run this notebook, you will need to install: pandas, openai, transformers, plotly, matplotlib, scikit-learn, torch (transformer dep), torchvision, and scipy.

In [3]:
# imports
import pandas as pd
import tiktoken

from openai.embeddings_utils import get_embedding


In [4]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191


In [7]:
# load & inspect dataset
input_datapath = "qa_project\\Reviews.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(5)


,Time,ProductId,UserId,Score,Summary,Text,combined
Id,,,,,,,
1,1303862400,B001E4KFG0,A3SGXH7AUHU8GW,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...
2,1346976000,B00813GRG4,A1D87F6ZCVE5NK,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...
3,1219017600,B000LQOCH0,ABXLMWJIXXAIN,4,"""Delight"" says it all",This is a confection that has been around a fe...,"Title: ""Delight"" says it all; Content: This is..."
4,1307923200,B000UA0QIQ,A395BORC6FGVXV,2,Cough Medicine,If you are looking for the secret ingredient i...,Title: Cough Medicine; Content: If you are loo...
5,1350777600,B006K2ZZ7K,A1UQRSCLF8GW1T,5,Great taffy,Great taffy at a great price. There was a wid...,Title: Great taffy; Content: Great taffy at a ...


In [8]:
# subsample to 1k most recent reviews and remove samples that are too long
top_n = 1000
df = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)


1000

In [10]:
import openai
## put your openai key here
openai.api_key = ""


## 2. Get embeddings and save them for future reuse

In [11]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))
df.to_csv("qa_project\\fine_food_reviews_with_embeddings_1k.csv")


In [14]:
df["embedding"]

Id
284932    [0.014864977449178696, -0.020280899479985237, ...
220697    [-0.023640209808945656, -0.011873619630932808,...
107908    [0.0002428288134979084, 0.005278090480715036, ...
107800    [0.010522752068936825, -0.013524581678211689, ...
205313    [0.014081825502216816, 0.0069235642440617085, ...
                                ...                        
7178      [-0.008826011791825294, -0.018647437915205956,...
401972    [-0.024472814053297043, -0.0175948329269886, 0...
462088    [-0.01685294136404991, -0.0051534585654735565,...
267549    [-0.010731690563261509, -0.01646377146244049, ...
542497    [-0.006057822611182928, -0.015015840530395508,...
Name: embedding, Length: 1000, dtype: object